# Start LangGraph


In [1]:
# !pip install langchain==0.2.0
# !pip install langchain-openai==0.1.7
# !pip install langchain-community==0.2.0
# !pip install langgraph==0.1.1
# !pip install langchain-chroma==0.1.1        

In [2]:
# pip install sqlalchemy

In [3]:
import os
import re
import json
import yaml
import pandas as pd
import psycopg2
from openai import OpenAI
from dotenv import load_dotenv

# Setup Environment Variables

In [4]:
# use file .env to get the API key
from dotenv import load_dotenv
load_dotenv()

import os

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY


# Build a Vector Database

In [5]:
from langchain_openai import OpenAIEmbeddings
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [6]:
from langchain.docstore.document import Document
import pandas as pd

companies_df = pd.read_csv("Data Kien/djia_companies_20250426.csv")
companies_df.head()

docs = [
    Document(
        page_content=row['description'],
        metadata={'symbol': row['symbol'], 'name': row['name'], 'sector': row['sector']}
    )
    for _, row in companies_df.iterrows()
]


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_docs = splitter.split_documents(docs)

from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model='text-embedding-3-small')


In [8]:
from langchain_chroma import Chroma

chroma_db = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embed_model,
    collection_name='djia_company_info',
    persist_directory="./djia_vector_db"
)
similarity_threshold_retriever  = chroma_db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.3})


In [9]:
query = "What does Apple do?"
results = similarity_threshold_retriever .invoke(query)
for r in results:
    print(r.metadata['name'], "->", r.page_content[:200])


Apple Inc. -> Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal 
Apple Inc. -> Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal 
Apple Inc. -> Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal 


## Create a Query Retrieval Grader

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# Tạo LLM với cấu hình Pydantic V2
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Parser
parser = StrOutputParser()

# Prompt system
SYS_PROMPT = """
You are an expert grader assessing relevance of a retrieved document to a user question.
Answer only 'yes' or 'no' depending on whether the document is relevant to the question.
"""

# Tạo prompt template
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", SYS_PROMPT),
    ("human", "Retrieved document:\n{document}\n\nUser question:\n{question}")
])

# Tạo chain xử lý
doc_grader = grade_prompt | llm | parser

In [11]:
query = "What does Apple do?"
top3_docs = similarity_threshold_retriever.invoke(query)
for doc in top3_docs:
	print(doc.page_content)
	print('GRADE:', doc_grader.invoke({
		"question": query,
		"document": doc.page_content
	}))
	print()

Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription


# Build a QA RAG Chain

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# Create RAG prompt for response generation
prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If no context is present or if you don't know the answer, just say that you don't know the answer.
Do not make up the answer unless it is there in the provided context.
Give a detailed answer and to the point answer with regard to the question.
Question:
{question}
Context:
{context}
Answer:
"""
prompt_template = ChatPromptTemplate.from_template(prompt)

# Initialize connection with gpt-4.1-mini
chatgpt = ChatOpenAI(model_name='gpt-4.1-mini', temperature=0)

# Used for separating context docs with new lines
def format_docs(docs):
	return "\n\n".join(doc.page_content for doc in docs)

# Create QA RAG chain
qa_rag_chain = (
	{
		"context": (itemgetter('context')
					|
					RunnableLambda(format_docs)),
		"question": itemgetter('question')
	}
	|
	prompt_template
	|
	chatgpt
	|
	StrOutputParser()
)

In [13]:
query = "What does Apple do?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke({"context": top3_docs, "question": query})
print(result)

Apple Inc. designs, manufactures, and markets a wide range of technology products and services worldwide. Its product lineup includes smartphones (iPhone), personal computers (Mac), multi-purpose tablets (iPad), and various wearables and accessories such as AirPods, Apple TV, Apple Watch, Beats products, and HomePod. Beyond hardware, Apple provides support services like AppleCare and cloud services. The company also operates digital platforms, including the App Store, which enables customers to discover and download applications and digital content such as books, music, videos, games, and podcasts. Additionally, Apple offers advertising services through third-party licensing and its own advertising platforms. It also provides several subscription-based services, including Apple Arcade (a game subscription service), Apple Fitness+ (a personalized fitness service), Apple Music (a curated music streaming service), and Apple News+ (a subscription news service).


In [14]:
query = "who won the champions league in 2024?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke({"context": top3_docs, "question": query})
print(result)

c:\Users\trung\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\vectorstores\base.py:1045: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'name': 'Coca-Cola Company (The)', 'sector': 'Consumer Defensive', 'symbol': 'KO'}, page_content='Powerade, Topo Chico, Core Power, Del Valle, fairlife, innocent, Maaza, Minute Maid, Minute Maid Pulpy, and Simply brands. It operates through a network of independent bottling partners, distributors, wholesalers, and retailers, as well as through bottling and distribution operators. The company was founded in 1886 and is headquartered in Atlanta, Georgia.'), -0.233447667864902), (Document(metadata={'name': 'Coca-Cola Company (The)', 'sector': 'Consumer Defensive', 'symbol': 'KO'}, page_content='Powerade, Topo Chico, Core Power, Del Valle, fairlife, innocent, Maaza, Minute Maid, Minute Maid Pulpy, and Simply brands. It operates through a network of independent bottling partners, distributors, whole

I don't know the answer.


# Create a Query Rephraser

In [15]:
# LLM for question rewriting
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
# Prompt template for rewriting
SYS_PROMPT = """Act as a question re-writer and perform the following task:
				 - Convert the following input question to a better version that is optimized for web search.
				 - When re-writing, look at the input question and try to reason about the underlying semantic intent / meaning.
			 """
re_write_prompt = ChatPromptTemplate.from_messages(
	[
		("system", SYS_PROMPT),
		("human", """Here is the initial question:
					 {question}
					 Formulate an improved question.
				  """)
	]
)

# Create rephraser chain
question_rewriter = (re_write_prompt|llm|StrOutputParser())

In [16]:
query = "who won the champions league in 2024?"
question_rewriter.invoke({"question": query})

'Who was the winner of the UEFA Champions League in 2024?'

# Load Web Search Tool

In [17]:
# %pip install langchain-core

In [18]:
# Load Web Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults
tv_search = TavilySearchResults(max_results=3, search_depth='advanced', max_tokens=10000)

# SQL Query

In [19]:
load_dotenv()

DB_CONFIG = {
    "dbname": os.getenv("DBNAME") or os.getenv("POSTGRES_DB"),
    "user": os.getenv("DBUSER") or os.getenv("POSTGRES_USER"),
    "password": os.getenv("DBPASSWORD") or os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("DBHOST") or "localhost",
    "port": int(os.getenv("DBPORT", 5432)),
    "sslmode": os.getenv("SSL_MODE", "require")
}


print(f"DB Config: {DB_CONFIG}")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def connect_to_database():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("Kết nối thành công đến PostgreSQL.")
        return conn
    except Exception as e:
        print(f"Lỗi kết nối cơ sở dữ liệu: {e}")
        return None

def get_schema_and_samples(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
        tables = cursor.fetchall()
        schema_info = {}
        for (table,) in tables:
            cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table}';")
            schema_info[table] = [{"column_name": col, "data_type": dtype} for col, dtype in cursor.fetchall()]
            cursor.execute(f'SELECT * FROM "{table}" LIMIT 3;')
            sample_rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]
            schema_info[f"{table}_samples"] = [dict(zip(colnames, row)) for row in sample_rows]
        cursor.close()
        return schema_info
    except Exception as e:
        return {"error": str(e)}

def generate_sql_query(user_question, schema_info=None):
    prompt = f"""
You are a PostgreSQL expert. You are working with a financial database containing two structured tables: "djia_prices" and "djia_companies".

IMPORTANT:
PostgreSQL is case-sensitive **only when using quoted identifiers**.
You MUST wrap column names in double quotes (e.g. "Ticker", "Close", "Date", etc.)
Do NOT use unquoted identifiers like Ticker or Close - they will cause errors.

Table 1: "djia_prices" - Daily stock trading data per company
- "Date" (TIMESTAMPTZ): The timestamp of the trading day.
- "Open" (FLOAT): Opening price of the stock on that day.
- "High" (FLOAT): Highest price reached during the trading day.
- "Low" (FLOAT): Lowest price of the stock during the trading day.
- "Close" (FLOAT): Closing price of the stock on that day.
- "Volume" (BIGINT): Number of shares traded on that day.
- "Dividends" (FLOAT): Dividend payout on that day, if any.
- "Stock_Splits" (FLOAT): Stock split ratio applied on that day (e.g. 2.0 = 2-for-1 split).
- "Ticker" (VARCHAR): The stock symbol of the company (e.g., AAPL, MSFT).

Table 2: "djia_companies" - Company profile information
- "symbol" (VARCHAR): The stock symbol matching the "Ticker" in "djia_prices".
- "name" (TEXT): Full name of the company.
- "sector" (TEXT): Main sector of operation (e.g., Technology, Healthcare).
- "industry" (TEXT): More specific industry classification (e.g., Consumer Electronics).
- "country" (TEXT): Country where the company is headquartered.
- "market_cap" (FLOAT): Market capitalization in USD.
- "pe_ratio" (FLOAT): Price-to-Earnings ratio of the company.
- "dividend_yield" (FLOAT): Annual dividend yield expressed as a percentage.
- "description" (TEXT): A short textual description of the company's operations.

Use this schema to write the most accurate and optimized PostgreSQL query to answer the following question.

Do NOT format the query in markdown or explain the result.
Return ONLY the raw SQL.

User Question:
{user_question}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        sql_query = response.choices[0].message.content
        sql_query = re.sub(r'^```sql', '', sql_query).strip('` \n')
        return sql_query
    except Exception as e:
        print(f"❌ Lỗi sinh SQL: {e}")
        return None


def execute_sql_query(conn, query):
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        print(f"❌ Lỗi thực thi SQL: {e}")
        return None

def run_chat(question: str):
    conn = connect_to_database()
    if conn is None:
        return

    schema_info = get_schema_and_samples(conn)
    sql_query = generate_sql_query(question, schema_info)

    if not sql_query:
        print("❌ Không thể sinh truy vấn SQL.")
        return

    print(f"\n🧠 SQL sinh ra:\n{sql_query}")
    results = execute_sql_query(conn, sql_query)

    if results is None:
        print("❌ Truy vấn lỗi.")
        return

    print("\n📊 Kết quả:")
    print(results.head(5))

    conn.close()

DB Config: {'dbname': 'postgres', 'user': 'postgres.qgoljxbwnlmvrutlylwp', 'password': 'WrWOSApFIXB2kI7E', 'host': 'aws-0-ap-southeast-1.pooler.supabase.com', 'port': 6543, 'sslmode': 'require'}


# Build Agentic RAG components

#### Graph State

In [20]:
from typing import List
from typing_extensions import TypedDict
class GraphState(TypedDict):
    question: str
    generation: str
    web_search_needed: str
    documents: List[str]
documents: List[str]

#### Retrieve function for retrieval from Vector DB

In [21]:
def retrieve(state):
	print("---RETRIEVAL FROM VECTOR DB---")
	question = state["question"]
	# Retrieval
	documents = similarity_threshold_retriever.invoke(question)
	return {"documents": documents, "question": question}

#### Grade documents

In [22]:
def grade_documents(state):
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state.get("documents", [])

    filtered_docs = []

    if documents:
        for d in documents:
            score = doc_grader.invoke(
                {"question": question, "document": d.page_content}
            )
            grade = score.strip().lower()

            if grade == "yes":
                print("✅ GRADE: DOCUMENT RELEVANT")
                filtered_docs.append(d)
            else:
                print("❌ GRADE: DOCUMENT NOT RELEVANT")
    else:
        print("Không có tài liệu nào từ vector DB.")

    return {
        **state,
        "documents": filtered_docs
    }


#### Rewrite query

In [23]:
def rewrite_query(state):
    print("---REWRITE QUERY---")
    question = state["question"]
    documents = state["documents"]
    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

In [24]:
# pip install tabulate

#### Use SQL

In [25]:
import pandas as pd
from langchain.schema import Document
from finance_metrics import load_formulas, identify_metric, get_required_fields, compute_metric
from plot_metric import plot_metric

def query_sql(state):
    print("---EXECUTE RAW SQL QUERY OR METRIC COMPUTATION---")
    question = state["question"]
    metadata = load_formulas()
    category, metric_name = identify_metric(question, metadata)

    # Kết nối đến DB
    conn = connect_to_database()
    if conn is None:
        raise ValueError("Không thể kết nối cơ sở dữ liệu.")

    # Nếu là chỉ số tài chính (FA hoặc TA)
    if metric_name:
        print(f"📌 Nhận diện chỉ số: {metric_name} ({category})")

        # --- TA: Vẽ biểu đồ từ dữ liệu giá ---
        if category == "TA":
            df = pd.read_sql(
                'SELECT "Date", "Close" FROM djia_prices WHERE "Ticker" = \'AAPL\' ORDER BY "Date" ASC LIMIT 100',
                conn
            )
            conn.close()
            df.rename(columns={"Date": "date", "Close": "price"}, inplace=True)

            image_base64 = plot_metric(metric_name, df)
            result_doc = Document(
                page_content=f"Biểu đồ {metric_name} cho AAPL:",
                metadata={"image_base64": image_base64}
            )
            return {
                "documents": state["documents"] + [result_doc],
                "question": question
            }

        # --- FA: Tính toán chỉ số từ bảng công ty ---
        elif category == "FA":
            required_fields = get_required_fields(category, metric_name, metadata)
            query = f"SELECT {', '.join(required_fields)} FROM djia_companies WHERE symbol = 'AAPL' LIMIT 1;"
            df = pd.read_sql(query, conn)
            conn.close()

            if df.empty:
                result_str = f"⚠️ Không tìm thấy đủ dữ liệu để tính {metric_name}."
            else:
                result = compute_metric(metric_name, df.iloc[0].to_dict())
                result_str = f"{metric_name} = {result}"

            return {
                "documents": state["documents"] + [Document(page_content=result_str)],
                "question": question
            }

    # Trường hợp không phải chỉ số → xử lý như SQL tự nhiên
    schema_info = get_schema_and_samples(conn)
    sql_query = generate_sql_query(question, schema_info)
    if not sql_query:
        conn.close()
        raise ValueError("Không thể sinh truy vấn SQL từ câu hỏi.")

    results = execute_sql_query(conn, sql_query)
    conn.close()

    # ⚠️ Nếu không có dữ liệu → yêu cầu fallback web search
    if results is None or results.empty:
        content = "⚠️ Không có kết quả từ truy vấn SQL."
        state["web_search_needed"] = "Yes"
    else:
        content = f"📊 Kết quả từ truy vấn SQL:\n\n{results.to_markdown(index=False)}"

    doc = Document(page_content=content)

    return {
        "documents": state["documents"] + [doc],
        "question": question,
        "sql_query": sql_query,
        "web_search_needed": state.get("web_search_needed", "No")  # vẫn duy trì trạng thái fallback
    }


#### decide merge or query sql

In [26]:
def decide_merge_or_sql(state):
    docs = state.get("documents", [])
    if len(docs) >= 1:
        return "merge_documents"
    return "query_sql"


#### merge_documents

In [27]:

def merge_documents(state):
    vector_docs = state.get("documents", [])
    sql_doc = state.get("sql_result", None)

    if sql_doc:
        merged_docs = vector_docs + [sql_doc]
    else:
        merged_docs = vector_docs

    return {
        **state,
        "documents": merged_docs
    }


#### Decide web search or generate answer

In [28]:
def decide_after_sql(state):
    if state.get("web_search_needed", "No") == "Yes":
        print("---DECISION: Missing SQL data → web search---")
        return "rewrite_query"
    
    print("---DECISION: Sufficient SQL data → generate answer---")
    return "generate_answer"


#### Web search

In [29]:
from langchain.schema import Document
def web_search(state):
    """
    Web search based on the re-written question.
    Args:
    state (dict): The current graph state
    Returns:
    state (dict): Updates documents key with appended web results
    """
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = tv_search.invoke(question)
    
    # Gỡ lỗi: kiểm tra cấu trúc trả về
    print("DOCS TYPE:", type(docs))
    print("FIRST ELEMENT TYPE:", type(docs[0]) if docs else "EMPTY")

    # Nếu docs là list of strings
    if isinstance(docs[0], str):
        web_content = "\n\n".join(docs)
    # Nếu docs là list of dicts
    elif isinstance(docs[0], dict) and "content" in docs[0]:
        web_content = "\n\n".join([d["content"] for d in docs])
    # Nếu docs là list of Document
    elif isinstance(docs[0], Document):
        web_content = "\n\n".join([d.page_content for d in docs])
    else:
        raise TypeError("Unsupported doc format")

    web_results = Document(page_content=web_content)
    documents.append(web_results)

    return {"documents": documents, "question": question}

#### Generate Answer

In [30]:
def generate_answer(state):
    print("---GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]
    # RAG generation
    generation = qa_rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question,"generation": generation}

# Build the Agent Graph with LangGraph

In [31]:
from langgraph.graph import StateGraph, END

# --- Build enhanced Agentic RAG with SQL-priority + Vector fallback + multi-hop merge ---
agentic_rag = StateGraph(GraphState)

# Add nodes
agentic_rag.add_node("retrieve", retrieve)
agentic_rag.add_node("grade_documents", grade_documents)
agentic_rag.add_node("merge_documents", merge_documents)  # New: combine vector + SQL docs if relevant
agentic_rag.add_node("query_sql", query_sql)              # Always query SQL early
agentic_rag.add_node("rewrite_query", rewrite_query)
agentic_rag.add_node("web_search", web_search)
agentic_rag.add_node("generate_answer", generate_answer)

# Entry point
agentic_rag.set_entry_point("retrieve")
agentic_rag.add_edge("retrieve", "grade_documents")

# Sau grading → nếu tài liệu vector đủ tốt thì merge luôn
agentic_rag.add_conditional_edges(
    "grade_documents",
    decide_merge_or_sql,  # → trả về: "merge_documents" hoặc "query_sql"
    {
        "merge_documents": "merge_documents",
        "query_sql": "query_sql"
    }
)

# Sau merge → luôn thực hiện SQL truy vấn
agentic_rag.add_edge("merge_documents", "query_sql")

# Sau query_sql → nếu đủ → generate, nếu thiếu → web search
agentic_rag.add_conditional_edges(
    "query_sql",
    decide_after_sql,
    {
        "generate_answer": "generate_answer",
        "rewrite_query": "rewrite_query"
    }
)

# Web search flow
agentic_rag.add_edge("rewrite_query", "web_search")
agentic_rag.add_edge("web_search", "generate_answer")

# END
agentic_rag.add_edge("generate_answer", END)

# Compile
agentic_rag = agentic_rag.compile()


In [32]:
print(type(agentic_rag))

<class 'langgraph.graph.state.CompiledStateGraph'>


In [33]:
# from IPython.display import Image, display, Markdown
# display(Image(agentic_rag.get_graph().draw_mermaid_png()))

# Test Agentic RAG System

In [34]:
from IPython.display import Markdown, display

In [35]:
query = "Which companies in the DJIA have a P/E ratio greater than 25?"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---


c:\Users\trung\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\vectorstores\base.py:1045: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'name': 'International Business Machines', 'sector': 'Technology', 'symbol': 'IBM'}, page_content="International Business Machines Corporation, together with its subsidiaries, provides integrated solutions and services in the United States, Europe, the Middle East, Africa, Asia Pacific, and internationally. It operates through Software, Consulting, Infrastructure, and Financing segments. The Software segment offers hybrid cloud and AI platforms that allows clients to realize their digital and AI transformations across the applications, data, and environments in which they operate. Its Consulting segment focuses on skills integration for strategy, experience, technology, and operations by domain and industry. The Infrastructure segment provides on-premises and cloud based server, and storage sol

---CHECK DOCUMENT RELEVANCE TO QUESTION---
Không có tài liệu nào từ vector DB.
---EXECUTE RAW SQL QUERY OR METRIC COMPUTATION---
Kết nối thành công đến PostgreSQL.


C:\Users\trung\AppData\Local\Temp\ipykernel_12536\2693450380.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


---DECISION: Sufficient SQL data → generate answer---
---GENERATE ANSWER---


The companies in the DJIA that have a P/E ratio greater than 25 are:

- Apple Inc.
- Amgen Inc.
- Salesforce, Inc.
- Walt Disney Company (The)
- Dow Inc.
- International Business Machines
- Coca-Cola Company (The)
- McDonald's Corporation
- Microsoft Corporation
- Procter & Gamble Company (The)
- Visa Inc.
- Walmart Inc.

These are the companies listed in the provided context as having a P/E ratio above 25.

In [36]:
query = "List companies whose average closing price in the last 90 days is above 200 and have a P/E ratio lower than 20."
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---


c:\Users\trung\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\vectorstores\base.py:1045: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'name': 'Dow Inc.', 'sector': 'Basic Materials', 'symbol': 'DOW'}, page_content='Dow Inc., through its subsidiaries, provides various materials science solutions for packaging, infrastructure, mobility, and consumer applications in the United States, Canada, Europe, the Middle East, Africa, India, the Asia Pacific, and Latin America. The company operates through Packaging & Specialty Plastics, Industrial Intermediates & Infrastructure, and Performance Materials & Coatings segments. The Packaging & Specialty Plastics segment provides ethylene, propylene, polyethylene, and aromatics products; and other ethylene derivatives, such as polyolefin elastomers, ethylene vinyl acetate, and ethylene propylene diene monomer rubber. The Industrial Intermediates & Infrastructure segment offers polyurethanes,

---CHECK DOCUMENT RELEVANCE TO QUESTION---
Không có tài liệu nào từ vector DB.
---EXECUTE RAW SQL QUERY OR METRIC COMPUTATION---
Kết nối thành công đến PostgreSQL.


C:\Users\trung\AppData\Local\Temp\ipykernel_12536\2693450380.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


---DECISION: Sufficient SQL data → generate answer---
---GENERATE ANSWER---


The companies whose average closing price in the last 90 days is above 200 and have a P/E ratio lower than 20 are:

1. Caterpillar, Inc. (CAT)  
   - Average Closing Price: 324.451  
   - P/E Ratio: 13.8917

2. UnitedHealth Group Incorporated (UNH)  
   - Average Closing Price: 502.269  
   - P/E Ratio: 17.5163

3. The Travelers Companies, Inc. (TRV)  
   - Average Closing Price: 253.19  
   - P/E Ratio: 14.1078

4. American Express Company (AXP)  
   - Average Closing Price: 271.41  
   - P/E Ratio: 18.4794

5. JP Morgan Chase & Co. (JPM)  
   - Average Closing Price: 243.498  
   - P/E Ratio: 11.9446

6. Goldman Sachs Group, Inc. (GS)  
   - Average Closing Price: 559.867  
   - P/E Ratio: 12.6418

All these companies meet the criteria of having an average closing price above 200 and a P/E ratio below 20.

1. Câu hỏi tập trung vào chỉ số tài chính (EPS, P/E, dividend, market cap):
- Which companies in the DJIA have a P/E ratio greater than 25?

- List the top 5 companies with the highest market capitalization and show their dividend yield and P/E ratio.

- What is the average P/E ratio of companies in the Technology sector?

- Show all companies with a dividend yield greater than 3% and market cap over 500 billion USD.

- Find all companies where the P/E ratio is less than the average P/E ratio across all DJIA companies.

2. Câu hỏi kết hợp djia_prices + djia_companies (gợi ý về EPS qua Close):
- Calculate the average closing price of Apple (AAPL) over the past 30 days.

- For each company, calculate the maximum daily trading volume in the last 6 months.

- List companies whose average closing price in the last 90 days is above 200 and have a P/E ratio lower than 20.

🔁 Luồng thực thi từng bước
🟩 1. retrieve
Truy xuất tài liệu từ vector DB (Chroma)

Câu hỏi người dùng được dùng để tìm các đoạn văn mô tả công ty, ngành, hoặc kiến thức liên quan

🟦 2. grade_documents
Dùng LLM để đánh giá mức độ liên quan của các đoạn tài liệu tìm được

Nếu đủ liên quan, hệ thống đánh dấu là tài liệu có thể dùng để trả lời

🔁 3. Nhánh rẽ decide_merge_or_sql:
Nếu tài liệu liên quan tốt → chuyển sang merge_documents

Nếu tài liệu yếu / không đủ → đi thẳng đến query_sql để truy vấn dữ liệu có cấu trúc

🔷 4. merge_documents (mới thêm)
Kết hợp tài liệu vector (phi cấu trúc) với kết quả SQL (cấu trúc) để tạo đầu vào toàn diện cho LLM

Hữu ích trong trường hợp: "Giá cổ phiếu trong 30 ngày qua và công ty hoạt động trong lĩnh vực nào?"

🟨 5. query_sql
Thực hiện truy vấn dữ liệu bảng từ PostgreSQL (ví dụ: djia_prices, djia_companies)

Tự động xử lý các truy vấn liên quan đến chỉ số FA/TA nếu phát hiện (P/E, RSI, v.v.)

🔁 6. Nhánh rẽ decide_after_sql:
Nếu dữ liệu bảng đủ để trả lời → chuyển đến generate_answer

Nếu dữ liệu thiếu hoặc không tìm thấy → sang rewrite_query để chuẩn bị cho web search

🌐 7. rewrite_query → web_search
LLM viết lại câu hỏi để tối ưu hóa kết quả khi gửi lên API tìm kiếm (Tavily)

Sau đó thực hiện web_search và lấy thông tin từ web

🧠 8. generate_answer
LLM tổng hợp các thông tin từ SQL, vector, và/hoặc web để sinh ra câu trả lời cuối cùng

✅ 9. END
Trả kết quả về cho người dùng